## SpanBERT for coreference resolution

documentation: https://github.com/mandarjoshi90/coref

In [1]:
!pwd

/home/akvallapuram/FYP-19-20/notebook/modelling/coref_resolution


In [2]:
# as per the documentation
!export data_dir='../../data'

### Preprocessing

Need to process a document to be resolved into jsonline files as described in the documentation.

In [3]:
import sys
# get sample article -> Not Working!
# sys.path.append('../../../')
# from src.preparation.data_loading import read_dossier

In [4]:
dos = "1. Speaking to a trusted compatriot in June 2016 sources A and B, a senior Russian Foreign \
Ministry figure and a former top level Russian intelligence officer still active inside the Kremlin \
respectively, the Russian authorities had been cultivating and supporting US Republican presidential \
candidate, Donald TRUMP for at least 5 years. Source B asserted that the TRUMP operation was both \
supported and directed by Russian President Vladimir PUTIN. Its aim was to sow discord and disunity \
within the US itself, but more especially within the Transatlantic alliance which was viewed as \
inimical to Russia's interests. Source C, a senior Russian financial official said the TRUMP \
operation should be seen in terms of PUTIN'S desire to return to Nineteenth Century 'Great Power' \
politics anchored upon countries' interests rather than the ideals-based international order \
established after World War Two.  S/he had overheard PUTIN talking in this way to close associates \
on several occasions. In terms of specifics, Source A confided that the Kremlin had been feeding TRUMP \
and his team valuable intelligence on his opponents, including Democratic presidential candidate \
Hillary CLINTON, for several years (see more below). This was confirmed by Source D, a close associate \
of TRUMP who had organized and managed his recent trips to Moscow who reported, also in June 2016, that \
this Russian intelligence had been “very helpful”. The Kremlin's cultivation operation on TRUMP also had \
comprised offering him various lucrative real estate develop me business deals in Russia, especially in \
relation to the ongoing 2018 World Cup soccer tournament. However, so far, for reasons unknown, TRUMP had \
not taken up any of these. However, there were other aspects to TRUMP's engagement with the Russian authorities. \
One which had borne fruit for them was to exploit TRUMP'S personal obsessions and sexual perversion in \
order to obtain suitable 'kompromat' (compromising material) on him. According to Source D, where s/he had \
been present, TRUMP's (perverted conduct in Moscow included hiring the presidential suite of the Ritz Carlton \
Hotel, where he knew President and Mrs OBAMA (whom he hated) had stayed on one of their official trips to Russia,\
and defiling the bed where they had slept by employing a number of prostitutes to perform a golden \
showers (urination) show in front of him. The hotel was known to be under FSB control with microphones \
and concealed cameras in all the main rooms to record anything they wanted to. The Moscow Ritz Carlton episode \
involving TRUMP reported above was confirmed by Source E,| who said that s/he and several of the staff were aware \
of it at the time and subsequently. S/he believed it had happened in 2013. Source E provided an introduction for \
a company ethnic Russian operative to Source F, a female staffer at the hotel when TRUMP had stayed there, who \
also confirmed the story. Speaking separately in June 2016, Source B [the former top level Russian intelligence \
officer) asserted that TRUMP's unorthodox behavior in Russia over the years had provided the authorities there \
with enough embarrassing material on the now Republican presidential candidate to be able to blackmail him if \
they so wished. 5. Asked about the Kremlin's reported intelligence feed to TRUMP over recent years and rumours \
about a Russian dossier of 'kompromat' on Hillary CLINTON (being circulated), Source B confirmed the file's \
existence. S/he confided in a trusted compatriot that it had been collated by Department K of the FSB for many \
years, dating back to her husband Bill's presidency, and comprised mainly eavesdropped conversations of various \
sorts rather than details/evidence of unorthodox or embarrassing behavior. Some of the conversations were from \
bugged comments CLINTON had made on her various trips to Russia and focused on things she had said which \
contradicted her current position on various issues. Others were most probably from phone intercepts. 6. Continuing \
on this theme, Source G, a senior Kremlin official, confided that the CLINTON dossier was controlled exclusively \
by chief Kremlin spokesman, Dmitriy PESKOV, who was responsible for compiling/handling it on the explicit \
instructions of PUTIN himself. The dossier however had not as yet been made available abroad, including to TRUMP \
or his campaign team. At present it was unclear what PUTIN'S intentions were in this regard."

In [5]:
# BERT Tokenizer
sys.path.append('../../../models/coref/')
from bert import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file='../../../models/bert_large/vocab.txt', do_lower_case=False)

/home/akvallapuram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akvallapuram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akvallapuram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akvallapuram/anaconda3/lib/python3.6/site-packages/tensorf

In [6]:
sents = list(filter(lambda x: len(x) > 3, dos.split('.')))
sents = list(map(lambda x: tokenizer.tokenize(x.strip()) + ['.'], sents))
sents[0] = ["[CLS]"] + sents[0]
sents[-1].append("[SEP]")

In [7]:
def get_subtoken_map(sent_list):
    subtoken_map = list()
    i = -1
    for sent in sent_list:
        for t in range(len(sent)):
            if (not sent[t].startswith('#') \
            and sent[t-1] != "[CLS]") \
            and sent[t] != "[SEP]":
                i += 1
            
            subtoken_map.append(i)
    return subtoken_map


# [item for sublist in [[i]*len(sents[i]) for i in range(len(sents))] for item in sublist]
def get_sentence_map(sent_list):
    sent_map = list()
    for i in range(len(sent_list)):
        sent_map.extend([i]*len(sent_list[i]))
    
    return sent_map

In [8]:
# create jsonline file
data = dict()
data['clusters'] = []
data['doc_key'] = 'nw'
data['sentences'] = sents
data['speakers'] = list(map(lambda x: ""*len(x), sents)) # setting to No speaker for every subword at the moment
data['sentence_map'] = get_sentence_map(sents)
data['subtoken_map'] = get_subtoken_map(sents)

In [9]:
# check the subtoken_map: correct
print(list(zip(sents[0], data['subtoken_map'][:len(sents[0])])))

[('[CLS]', 0), ('Speaking', 0), ('to', 1), ('a', 2), ('trusted', 3), ('com', 4), ('##pa', 4), ('##tri', 4), ('##ot', 4), ('in', 5), ('June', 6), ('2016', 7), ('sources', 8), ('A', 9), ('and', 10), ('B', 11), (',', 12), ('a', 13), ('senior', 14), ('Russian', 15), ('Foreign', 16), ('Ministry', 17), ('figure', 18), ('and', 19), ('a', 20), ('former', 21), ('top', 22), ('level', 23), ('Russian', 24), ('intelligence', 25), ('officer', 26), ('still', 27), ('active', 28), ('inside', 29), ('the', 30), ('K', 31), ('##rem', 31), ('##lin', 31), ('respectively', 32), (',', 33), ('the', 34), ('Russian', 35), ('authorities', 36), ('had', 37), ('been', 38), ('cult', 39), ('##ivating', 39), ('and', 40), ('supporting', 41), ('US', 42), ('Republican', 43), ('presidential', 44), ('candidate', 45), (',', 46), ('Donald', 47), ('T', 48), ('##R', 48), ('##UM', 48), ('##P', 48), ('for', 49), ('at', 50), ('least', 51), ('5', 52), ('years', 53), ('.', 54)]


In [10]:
# write data to a jsonline format file
import json

In [11]:
with open('Dossier_for_spanBERT_coref.jsonlines', 'w') as f:
    f.write(json.dumps(data))

After preprocessing, ran the following in the `models/coref/` directory:

```
    GPU=0 python predict.py bert_large ../../notebook/modelling/coref_resolution/Dossier_for_spanBERT_coref.jsonlines Dossier_spanBERT_predictions.jsonlines
```

which produced the following error:

```
Traceback (most recent call last):
  File "predict.py", line 33, in <module>
    _, _, _, top_span_starts, top_span_ends, top_antecedents, top_antecedent_scores = session.run(model.predictions, feed_dict=feed_dict)
  File "/home/akvallapuram/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 929, in run
    run_metadata_ptr)
  File "/home/akvallapuram/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1121, in _run
    np_val = np.asarray(subfeed_val, dtype=subfeed_dtype)
  File "/home/akvallapuram/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py", line 85, in asarray
    return array(a, dtype, copy=False, order=order)
ValueError: setting an array element with a sequence.
```